<a href="https://colab.research.google.com/github/dat810/tt2_things/blob/main/present_panic/bomb_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prestige Event Bomb Game

Exploring the following:
- Average scissors needed per board
- Average boards completed for the event

So that you are prepared for the RNG 🤣

## Setting Up
Loading libraries

In [4]:
import random
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp
import os
import requests

Loading the dataset from `TT2_CSV` by rawrzcookie (as of 7.5.2)

In [9]:
!git clone https://github.com/rawrzcookie/TT2_CSV.git

fatal: destination path 'TT2_CSV' already exists and is not an empty directory.


In [8]:
!cp TT2_CSV/csv/HolidayEventBombGameLevelInfo.csv .

Defining a Board

In [6]:
class Board:
    def __init__(self,dim,bombs,target,diff_idx):
        self.dim = dim
        self.bombs = bombs
        self.target = target
        self.boxes = self.dim[0]*self.dim[1]

        idx_to_diff = {
            0: 'Easy', 1: 'Hard', 2: 'Impossible'
        }
        self.diff = idx_to_diff.get(diff_idx)


    def __str__(self):
        return (
            f"Board size: {self.dim[0]}x{self.dim[1]}\n"
            f"Bombs count: {self.bombs}\n"
            f"Streak needed: {self.target}\n"
            f"Difficulty: {self.diff}"
        )

Defining one board simulation

In [ ]:
%%writefile present_panic_sim.c
#include <stdlib.h>
#include <time.h>

int present_panic_sim (int p, int t, int target) {
    int g = p - t;
    int total_scissors = 0;
    int streak = 0;
    int *board = (int *)malloc(p*sizeof(int));
    int i;

    // Initialize the board: 0 for trolls, 1 for ginger
    for (i=0;i<t;i++) {
        board[i] = 0; // troll
    }
    for (i=t;i<p;i++) {
        board[i] = 1; // ginger
    }

    srand((unsigned int)time(NULL)+rand());

    while (streak < target) {
        int index = rand() % p;
        total_scissors++;

        if (board[index] == 1) { // ginger
            streak++;
            board[index] = board[--p]; // Remove ginger and shrink board
        } else { // troll
            streak = 0;
            t--;
            p = t + g; // Reset board
            for (i = 0; i < t; i++) board[i] = 0; // troll
            for (i = t; i < p; i++) board[i] = 1; // ginger
        }
    }

    free(board);
    return total_scissors;
}

## Average Scissors Needed per Board

## Average Boards Completed in an Event

## Notebook Information
- Notebook content created by dat
- Dataset from rawrzcookie, who also fetched the CSV files here: https://github.com/rawrzcookie/TT2_CSV